In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.datasets import cifar10 

In [2]:
import tensorflow.keras as K

In [3]:
from keras.applications.vgg16 import VGG16

In [4]:
from keras.utils import np_utils 

In [5]:
def preprocess_data(X, Y):
        """ This method has the preprocess to train a model """
        X = X.astype('float32')
        X_p = K.applications.vgg16.preprocess_input(X)
        Y_p = K.utils.to_categorical(Y, 10)
        return(X_p, Y_p)

In [6]:
(Xt, Yt), (X, Y) = cifar10.load_data()

In [7]:
X_p, Y_p = preprocess_data(Xt, Yt)
Xv_p, Yv_p = preprocess_data(X, Y)

In [8]:
base_model = VGG16(include_top=False,weights='imagenet',pooling='avg')


In [9]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Input,UpSampling2D

In [10]:
model3= Sequential()
model3.add(UpSampling2D())
model3.add(base_model)
model3.add(Flatten())
model3.add(Dense(512, activation=('relu'))) 
model3.add(Dropout(0.2))
model3.add(Dense(256, activation=('relu')))
model3.add(Dropout(0.2))
model3.add(Dense(10, activation=('softmax')))

In [12]:
model3.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x000001E368AC2F70>>

In [13]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [15]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
''''
ModelCheckpoint helps us to save the model by monitoring a specific parameter of the model. In this case I am monitoring 
validation accuracy by passing val_acc to ModelCheckpoint. The model will only be saved to disk if the validation accuracy 
of the model in current epoch is greater than what it was in the last epoch.

In [16]:
 model3.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model3.fit(x=X_p, y=Y_p,
              batch_size=128,
              validation_data=(Xv_p, Yv_p),
              epochs=10, shuffle=True,
              callbacks=[checkpoint,early],
              verbose=1
              )

Epoch 1/10
  7/391 [..............................] - ETA: 58:05 - loss: 7.0288 - accuracy: 0.0826